This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 325.49788774, ...,   0.        ,
        47.20790294,  23.809066  ]), array([219.14289644, 161.4705311 , 285.23022195, ...,  64.21264158,
        56.02046287,  46.33945456]), OrderedDict([('R2', 0.7727360467698481), ('root_mean_squared_error', 44.91038692115582), ('mean_absolute_error', 31.121798542384816), ('rmse_over_stdev', 0.48073259073861185)]), None)
test_quad = (array([2.02092457e+02, 2.45647989e+02, 2.39214774e+02, 2.29790128e+02,
       3.44799275e+02, 0.00000000e+00, 2.53584855e+02, 7.37475037e+01,
       6.47667857e+01, 1.46594936e+02, 1.40876460e+02, 1.40791488e+02,
       1.14194239e+02, 9.96717956e+01, 1.86835775e+02, 1.81659746e+02,
       7.15466155e+01, 7.75902143e+01, 6.73902194e+01, 1.13270942e+02,
       1.02665208e+02, 1.27306711e+02, 9.93875156e+01, 1.71809961e+02,
       1.44746721e+02, 1.22487016e+02, 1.90002071e+02, 1.90460889e+02,
       1.53899276e+02, 3.02985863e+02, 2.29722452e+02, 1.24022947e+02,
       1.19657588e+02, 9.33858524e+01, 1.65793652e+02, 1.74631224e+02,
       2.35011420e+01, 9.51442246e+01, 2.70007913e+02, 6.03566925e+01,
       1.00971120e+02, 8.07377667e+01, 8.42025056e+01, 8.95029615e+01,
       1.13135302e+02, 1.27681884e+02, 3.47342257e+01, 7.58782445e+01,
       4.93432857e+01, 7.48766194e+00, 0.00000000e+00, 0.00000000e+00,
       2.61724803e+02, 5.35777138e+01, 0.00000000e+00, 1.94608833e+01,
       5.64799569e+01, 0.00000000e+00, 1.30861416e+02, 0.00000000e+00,
       7.47720530e+01, 1.19778837e+02, 2.18946397e+02, 2.53560446e+02,
       2.94893175e+02, 1.17650530e+02, 5.95521104e+01, 0.00000000e+00,
       4.11709055e+01, 5.57153253e+01, 0.00000000e+00, 4.51430137e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.60810735e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.39741468e+01, 8.59609736e+01, 8.92356506e+01,
       1.73373103e+01, 1.74713200e+00, 0.00000000e+00, 3.49408401e+01,
       4.33637207e+01, 3.27899307e+01, 6.50312714e+01, 4.98477351e+01,
       4.02646946e+01, 4.39956935e+01, 2.32209209e+02, 1.61615856e+02,
       0.00000000e+00, 1.28935952e+02, 3.04689791e+01, 0.00000000e+00,
       0.00000000e+00, 9.92490625e-01, 0.00000000e+00, 9.52352009e+00,
       0.00000000e+00, 0.00000000e+00, 4.44184734e+01, 0.00000000e+00,
       3.45198214e+01, 3.41493981e+01, 3.55034057e+01, 6.29442201e+01,
       1.18542973e+02, 2.88123722e+01, 3.98312067e+01, 7.96465197e+01,
       3.66852424e+01, 3.51583952e+01, 2.82257425e+01, 3.61795100e+01,
       7.33952221e+01, 1.04542615e+02, 3.62570480e+01, 1.19114168e+02,
       8.04591080e+01, 1.23754016e+02, 1.26886312e+02, 4.68034609e+02,
       2.11720874e+01, 5.24822446e+01, 1.26881899e+02, 8.98167378e+01,
       1.24480773e+01, 3.29412986e+01, 9.34258071e+01, 1.71956703e+02,
       1.14554644e+02, 1.09033411e+02, 1.06702335e+02, 0.00000000e+00,
       2.07515714e+02, 3.99617942e+01, 0.00000000e+00, 2.02664553e+02,
       3.15202492e+01, 2.02283948e+02, 3.21989575e+02, 1.28448866e+02,
       2.11476824e+02, 6.24940495e+01, 0.00000000e+00, 0.00000000e+00,
       4.74586631e+01, 3.61695636e+01, 2.93008151e+01, 2.24196124e+01,
       5.14231024e+01, 0.00000000e+00, 5.44675418e+01, 0.00000000e+00,
       0.00000000e+00, 4.48488874e+01, 4.49236078e+01, 4.75455355e+01,
       4.45494170e+01, 6.45192863e+01, 4.42032278e+01, 2.62753383e+01,
       4.63304068e+01, 5.83323071e+01, 1.29976759e+02, 5.67330262e+01,
       3.96932977e+01, 4.16645268e+01, 4.52879705e+01, 4.43244216e+01,
       5.20549201e+01, 0.00000000e+00, 1.62892176e+02, 1.02151271e+02,
       9.46538849e+01, 0.00000000e+00, 6.07251297e+01, 3.61816245e+01,
       4.31287284e+01, 4.95815034e+01, 2.44136176e+01, 3.11645320e+01,
       1.10716943e+02, 5.48277307e+01, 1.98086046e+01, 6.89092871e+01,
       3.57280662e+01, 3.26110250e+01, 7.05592531e+00, 3.69821609e+00,
       5.31766390e-01, 1.12780820e+01, 5.20671817e+00, 1.85197855e+02,
       2.65270592e+02, 1.89093084e+02, 1.32871922e+02, 2.34660651e+02,
       1.86400275e+02, 2.13314660e+02, 2.76006928e+02, 1.14272223e+02,
       2.52767162e+02, 2.30118336e+02, 1.29607780e+02, 1.86026530e+02,
       1.18103266e+02, 1.89952208e+02, 1.24477908e+02, 2.44148911e+02,
       1.39013581e+02, 1.55694637e+02, 2.35409225e+02, 1.70627481e+02,
       2.18442616e+02, 1.52280199e+02, 1.40405957e+02, 1.29342617e+02,
       8.30462436e+01, 1.56136057e+02, 2.67340580e+02, 3.67183228e+02,
       3.62461054e+02, 1.77346156e+02, 1.91512363e+02, 2.37992715e+02,
       3.09856621e+02, 1.89648365e+02, 3.20529825e+02, 3.49453381e+02,
       3.18399908e+02, 2.32699819e+02, 1.01887892e+02, 5.98511381e+01,
       1.29319114e+02, 1.56185442e+02, 7.04378967e+01, 1.10209088e+02,
       9.77091114e+01, 1.51356144e+02, 1.77317832e+02, 1.84516539e+02,
       2.06610134e+02, 1.87885255e+02, 1.00763646e+02, 1.92010418e+02,
       9.48376021e+01, 2.34968862e+02, 1.54707611e+02, 2.40576558e+02,
       3.64707101e+02, 1.86717789e+02, 3.32212137e+02, 2.58640047e+02,
       3.17356871e+02, 2.30917642e+02, 8.95779086e+01, 6.56442278e+01,
       2.44800345e+02, 1.62784213e+02, 5.46384029e+01, 5.41587672e+01,
       1.32392433e+02, 5.31282447e+01, 1.60434807e+02, 7.28852659e+01,
       1.84330790e+02, 2.72871580e+02, 3.24945084e+02, 1.78519868e+02,
       1.84420825e+02, 1.44135045e+02, 1.48340548e+02, 1.91404536e+02,
       2.00760493e+02, 3.04851246e+02, 2.87755496e+02, 8.45337300e+02,
       9.50227906e+02, 1.42416121e+02, 1.20158300e+02, 6.77164974e+01,
       1.38844996e+02, 9.27983292e+01, 9.54271066e+01, 1.26955312e+02,
       1.22028582e+02, 1.89420372e+02, 9.04495542e+01, 8.32175656e+01,
       8.35342519e+01, 6.89800768e+01, 1.25312626e+02, 1.89038755e+01,
       2.72145096e+01, 2.91372971e+01, 6.07829098e+01, 1.12909303e+02,
       6.88773537e+01, 4.85930390e+01, 5.83780479e+01, 1.35840083e+02,
       1.84215977e+02, 1.84779292e+02, 1.63409286e+02, 1.54672899e+02,
       1.92128669e+02, 2.00391998e+02, 1.47483468e+02, 7.73860277e+01,
       9.06089707e+01, 1.14754340e+02, 1.13060644e+02, 1.35954460e+02,
       1.37791901e+02, 2.96630230e+02, 1.13092108e+02, 1.84901449e+02,
       1.88514713e+02, 1.85051275e+02, 1.56027039e+02, 2.08035658e+02,
       1.17143169e+02, 1.10864718e+02, 1.09297300e+02, 2.16122195e+02,
       1.41965660e+02, 2.44717542e+02, 6.97752946e+01, 1.07783426e+02,
       3.71426186e+01, 5.65113296e+01, 6.69909409e+01, 3.33198729e+01,
       3.48221974e+01, 1.97610067e+02, 2.15562064e+01, 4.45796202e+00,
       5.97413320e+01, 1.09582964e+02, 1.12949106e+02, 1.65733079e+02,
       1.98665047e+02, 1.34244899e+02, 2.84417281e+01, 9.98669764e+01,
       1.07860026e+02, 1.92009283e+02, 1.09972952e+02, 1.82449930e+02,
       8.45812329e+01, 0.00000000e+00, 9.82564416e+01, 4.95323894e+01,
       9.38423029e+01, 5.41514922e+01, 5.32856309e+01, 4.94793928e+01,
       2.83696735e+01, 0.00000000e+00, 7.43588963e+01, 3.97951736e+01,
       0.00000000e+00, 2.55044144e+00, 7.99376675e+00, 1.42008055e+01,
       3.41003692e+01, 0.00000000e+00, 2.77992144e+01, 1.41787474e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.81498187e+01,
       2.01387280e+01, 7.26501274e+01, 7.90798034e+01, 7.72377141e+01,
       1.52154945e+02, 9.16707430e+01, 1.66797820e+02, 3.81742435e+01,
       0.00000000e+00, 3.88036349e+01, 8.61157898e+01, 0.00000000e+00,
       8.72670422e+01, 8.26202450e+01, 7.33023683e+01, 5.33006835e+01,
       1.77019953e+01, 3.76093386e+01, 0.00000000e+00, 0.00000000e+00,
       2.80724115e+01, 3.07458097e+01, 3.05960388e+01, 8.01722752e+01,
       1.20865840e+02, 5.40260430e+01, 2.84167104e+01, 3.87785474e+01,
       4.66982172e+01, 4.53272821e+01, 3.34624193e+01, 9.28082416e+01,
       5.43341092e+01, 3.87540460e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.89029515e+02, 0.00000000e+00, 2.88957798e+01, 1.94101993e+01,
       0.00000000e+00]), array([140.14439003, 193.16509115, 226.46757628, 182.5719296 ,
       263.01960808, 218.39856712, 253.21828215, 151.19630401,
        99.27950075, 164.48741909, 162.18229175, 123.30184628,
       116.94505581,  92.95332446, 151.10934564, 209.71961177,
       178.4924962 , 129.02726524, 102.18375069, 138.86033743,
       155.86459486, 122.37198684, 134.73146699, 118.325171  ,
       167.55860789, 151.58824486, 185.5954657 , 192.37144048,
       173.85061464, 230.51979871, 302.76124064, 159.07233437,
       124.05542477, 131.81404893, 162.98421244, 158.83155706,
        68.42762797,  80.19619021, 140.45718033,  77.03146324,
        99.73560363, 122.94070845, 120.07982737, 107.21558544,
       135.82036317, 124.26812988,  83.23980146, 132.58582522,
       195.81063021, 150.84538337, 110.4529629 ,  87.89002766,
       245.17912811,  94.65908337,  34.00259064,  39.2469953 ,
        48.8652273 ,  58.62032065,  40.89678488,  59.03982952,
        48.29629329, 137.25316751, 166.2149922 , 222.33551817,
       243.79055081,  98.33966765,  54.63959044,  15.01269224,
        49.07143539,  80.63143424,   9.03766444,  45.16440292,
        31.68750544,  27.64180506,  35.73268211,  51.57428012,
        57.92587774,  21.35368217,  68.60274607,  46.12573757,
        20.20315299,   1.34587864,  39.16781905,  20.60125603,
        45.926699  ,  82.35884361,  50.28664576,  74.56898028,
        27.97171378,  26.80236537,  63.06485226,  34.67567983,
        49.76961113,  46.30465327,  46.240902  ,  28.67211644,
        29.76525659,  30.03330509, 228.49098113, 115.14901173,
        57.39329418, 114.46049098,  40.64847693,  13.81870399,
        30.71653   ,  34.14431459,  18.58336344,  13.33796874,
        35.02725899,  31.6728741 ,  56.53268556,  26.64919213,
        29.15610226,  46.18669303,  24.11538712,  32.31889473,
        57.16610277,  48.83777041,  57.29407747,  29.8370828 ,
        25.14023402,  36.31107115,  28.8935746 ,  23.95016827,
        24.60657521,  56.26384585,   6.28776916,  62.74458238,
        67.5882581 , 162.94796011, 161.65356813, 286.65627089,
        31.9636942 ,  38.40247446, 217.86760348,  80.89463301,
        41.09363912,  73.21876484, 191.3968226 , 125.420534  ,
       170.28029487, 189.47019009, 130.69002812,  80.81637587,
        44.88348883,  62.41130565,  11.26379196,  38.17532101,
        36.68800552,  45.80248631,  65.53594962,  74.99931337,
        82.51893352,  75.90367695,  16.45238877,  63.31399172,
        85.94702225,  46.76516919,  40.60576033,  58.02559304,
        50.72558453,  11.03729144,  53.19145763,  26.9945351 ,
         3.13095012,  44.78113908,  46.62057327,  24.22635157,
        43.14523204,  67.04401623,  59.80997665,  57.4473877 ,
        69.27335485,  83.54365228, 170.28606187,  52.52617581,
        48.24889979,  50.28503127,  18.01765195,  26.29211212,
        32.6994823 ,  -5.12596465, 174.05420928, 113.33619075,
        99.10435285,  34.5464723 ,  78.47672007,  37.76614827,
        47.01227331,  47.90784817,  29.18427503,  74.09827066,
        27.70310325,  38.70693731,   6.94292199,  23.71733164,
         3.7323524 ,  34.64442138,  23.74301172,  32.54244489,
        23.39026411,  21.96225933,  10.71886009, 155.2808088 ,
       306.04578896, 178.15447318, 145.03549445, 221.10426423,
       159.51144545, 154.40856595, 173.32402853, 124.76118706,
       304.00881908, 165.36510634, 182.41008684, 181.84850538,
       160.24541632, 192.30055343,  99.66603257, 107.90511496,
       165.49276445, 205.91394878, 233.93573353, 222.34500606,
       190.06872096, 100.02161704,  94.4117736 , 131.60080938,
        72.15482278, 159.76019733, 309.99057042, 274.72685081,
       256.11137744, 184.98270318, 207.28790121, 300.6877928 ,
       284.17805759, 193.34244592, 225.95233891, 251.6814998 ,
       265.42352137, 211.36121135,  88.06437237,  83.6540565 ,
        88.86257537, 158.83724502,  86.25508136,  88.40695517,
        79.77417292, 154.65089679, 173.9799869 , 190.30421508,
       145.77799854,  94.69469948,  66.71785002, 141.26497153,
        69.39841328, 155.63327778, 139.81812838, 169.31362915,
       307.26793219, 191.07862054, 307.82862398, 227.33858929,
       285.97403819, 182.40909046,  87.69660354,  80.79162501,
       257.53896469, 141.07865557,  91.22865794,  78.04143783,
       123.08424038,  67.68131453, 159.81160186, 111.03802291,
       154.27239057, 310.57063465, 307.89377021, 202.37811305,
       217.63879996, 185.62922464, 167.65447821, 205.05118653,
       221.8041679 , 234.25006786, 202.55419738, 402.3282975 ,
       482.8619869 , 137.17135144, 136.61389422, 121.67067858,
       110.63634689, 117.0422309 , 125.93391194, 142.95072882,
       118.67372758, 126.81611525, 124.50938091, 123.80930934,
       113.5192761 , 131.81569846, 150.20970978,  73.65729641,
        47.49549359,  39.6975934 , 104.43444027, 102.56168393,
        72.67547775, 107.85609238,  47.07888272, 233.52926423,
       189.0619339 , 158.24124713, 173.57480438, 163.08798233,
       170.56807798, 188.90780627, 126.45076881, 125.96640879,
       124.32307684, 153.18596619, 116.9367706 , 155.31289518,
       147.67325899, 315.34540862, 150.8138746 , 204.19743304,
       195.66584478, 116.72836563, 115.28266098, 150.80781806,
       126.65597844, 104.44703623, 105.23530261, 195.1244338 ,
       118.52695025, 198.8938061 , 105.06690287, 149.58970046,
        62.01719686,  63.60123794,  66.90226775,  36.69812847,
        37.11275542,  92.03120952,  48.9203583 ,  33.12252214,
        57.11572352,  95.5620043 ,  84.29341926, 116.89451345,
       151.56906178, 107.93173326,  27.40569139,  85.62206467,
        97.57239803, 131.62553942,  85.39427472, 152.54859689,
        86.11022362,  10.1509923 ,  54.44678174,  79.01306767,
        53.83420245,  45.25919835,  41.89636241,  56.98541722,
        39.27485959,  50.40920717,  54.44822313,  47.05824389,
        32.88896997,   3.73085498,  23.02841812,  21.63454039,
        44.60519804,   1.38235398,  31.00863104,  22.90332737,
        23.60282523,  38.40325572,  49.43878934,  54.32164327,
        27.64242806,  42.29642196,  49.0379636 ,  39.48553894,
       104.45288578,  43.01169324,  61.97975632,  34.65592171,
        31.84465105,  28.5622229 , 102.13481752,  14.25695775,
       105.38392366,  90.13776731,  97.72376163,  75.34442966,
        63.1136736 ,  62.85529402,  21.016681  ,  62.64798694,
        17.33344987,  20.75991793,  16.80296098,  51.43169583,
        90.91765422,  35.41501521,  15.07002272,  29.3055129 ,
        28.93611314,  26.0440777 ,  27.28110454,  33.61386481,
        32.69985245,  23.50309566,  18.06889144,  11.43705358,
        19.56359066,  10.70573169,  20.99789249,   9.20928459,
        70.64463976,  24.73710894,  36.37625696,  22.13366132,
         5.14062113]), OrderedDict([('R2', 0.7168016348410974), ('root_mean_squared_error', 55.74094257624111), ('mean_absolute_error', 33.96418149622039), ('rmse_over_stdev', 0.5966657063527909)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/split_7'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
